In [13]:
from datasets import load_dataset

In [14]:
data = load_dataset("lmms-lab/LiveBench", "2024-07")

Generating test split: 100%|██████████| 404/404 [00:01<00:00, 364.97 examples/s]


In [15]:
import pandas as pd

df = data["test"].to_pandas()
df = df[df["checker"].notna()]

In [27]:
df["subtask"].value_counts()

subtask
Contextual Analysis     103
Deeper Implications      97
Broader Implications     79
Basic Understanding      59
Further Insights         55
Name: count, dtype: int64

In [18]:
df = df.sample(frac=1).reset_index(drop=True)

In [28]:
df = df[df["id"] != 35]

In [29]:
top_50_per_subtask = df.groupby("subtask").apply(lambda x: x.nlargest(50, "score"))
top_50_per_subtask.reset_index(drop=True, inplace=True)
len(top_50_per_subtask)

/tmp/ipykernel_425178/992929014.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_50_per_subtask = df.groupby("subtask").apply(lambda x: x.nlargest(50, "score"))


250

In [30]:
top_50_per_subtask["subtask"].value_counts()

subtask
Basic Understanding     50
Broader Implications    50
Contextual Analysis     50
Deeper Implications     50
Further Insights        50
Name: count, dtype: int64

In [31]:
import numpy as np

np.mean(top_50_per_subtask["score"])

9.128

In [32]:
from datasets import Dataset, Features
import datasets


def gen():
    for d in top_50_per_subtask:
        yield d


data = Dataset.from_pandas(
    top_50_per_subtask,
    features=Features(
        {
            "id": datasets.Value("int32"),
            "images": datasets.Sequence(datasets.Image()),
            "website": datasets.Value("string"),
            "question": datasets.Value("string"),
            "answer": datasets.Value("string"),
            "criteria": datasets.Value("string"),
            "subtask": datasets.Value("string"),
            "data_generator": datasets.Value("string"),
            "checker": datasets.Value("string"),
            "date_time": datasets.Value("string"),
            "screen_shoter": datasets.Value("string"),
            "screen_size": datasets.Value("string"),
            "score": datasets.Value("int32"),
            "reason": datasets.Value("string"),
            "scorer_name": datasets.Value("string"),
        }
    ),
)

In [33]:
data

Dataset({
    features: ['id', 'images', 'website', 'question', 'answer', 'criteria', 'subtask', 'data_generator', 'checker', 'date_time', 'screen_shoter', 'screen_size', 'score', 'reason', 'scorer_name'],
    num_rows: 250
})

In [34]:
data.features

{'id': Value(dtype='int32', id=None),
 'images': Sequence(feature=Image(mode=None, decode=True, id=None), length=-1, id=None),
 'website': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None),
 'criteria': Value(dtype='string', id=None),
 'subtask': Value(dtype='string', id=None),
 'data_generator': Value(dtype='string', id=None),
 'checker': Value(dtype='string', id=None),
 'date_time': Value(dtype='string', id=None),
 'screen_shoter': Value(dtype='string', id=None),
 'screen_size': Value(dtype='string', id=None),
 'score': Value(dtype='int32', id=None),
 'reason': Value(dtype='string', id=None),
 'scorer_name': Value(dtype='string', id=None)}

In [35]:
data.push_to_hub("lmms-lab/LiveBench", "2024-07", split="test")

Uploading the dataset shards: 100%|██████████| 1/1 [00:19<00:00, 19.84s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lmms-lab/LiveBench/commit/19dd279ffe05a6911e11f93a1e9699b6daf7be4e', commit_message='Upload dataset', commit_description='', oid='19dd279ffe05a6911e11f93a1e9699b6daf7be4e', pr_url=None, pr_revision=None, pr_num=None)